# Example of pipeline: extracting and transforming pdf file

### Load packages

In [1]:
# pip3 install nougat-ocr

In [2]:
%reload_ext autoreload
%autoreload 2

import sys
import pprint

sys.path.append(".")
sys.path.append("..")
sys.path.append("../..")

In [3]:
import os
import pandas as pd
from uniflow.pipeline import MultiFlowsPipeline
from uniflow.flow.config import PipelineConfig
from uniflow.flow.config import TransformOpenAIConfig, ExtractPDFConfig
from uniflow.op.model.model_config import OpenAIModelConfig, NougatModelConfig
from uniflow.op.prompt_schema import GuidedPrompt, Context

/opt/conda/envs/uniflow/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Prepare the input data

First, let's set current directory and input data directory, and load the raw data.

In [4]:
dir_cur = os.getcwd()
pdf_file = "nike-paper.pdf"
input_file = os.path.join(f"{dir_cur}/data/raw_input/", pdf_file)

data = [
    {"pdf": input_file},
]

### Define extract config using Nougat

In [5]:
extract_config = ExtractPDFConfig(
    model_config=NougatModelConfig(
        model_name = "0.1.0-small",
        batch_size = 1 # When batch_size>1, nougat will run on CUDA, otherwise it will run on CPU
    )
)

### Prepare sample prompts

We need to demonstrate sample prompts for LLM. We do this by giving a sample list of `Context` examples to the `GuidedPrompt` class.

In [6]:
guided_prompt = GuidedPrompt(
    examples=[Context(
        context="In 1948, Claude E. Shannon published A Mathematical Theory of\nCommunication (Shannon, 1948) establishing the theory of\ninformation. In his article, Shannon introduced the concept of\ninformation entropy for the first time. We will begin our journey here.""",
        question="Who published A Mathematical Theory of Communication in 1948?""",
        answer="Claude E. Shannon."""
    )]
)

### Define transform config

In this example, we will use the [OpenAIModelConfig](https://github.com/CambioML/uniflow/blob/main/uniflow/model/config.py#L17)'s default LLM to generate questions and answers.

In [7]:
transform_config = TransformOpenAIConfig(
    guided_prompt_template=guided_prompt,
    model_config=OpenAIModelConfig(
        response_format={"type": "json_object"}
    ),
)

### Use MultiFlowsPipeline

In [8]:
p = MultiFlowsPipeline(PipelineConfig(
    extract_config=extract_config,
    transform_config=transform_config,
))

/opt/conda/envs/uniflow/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [9]:
output = p.run(data)

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 13/13 [01:50<00:00,  8.53s/it]


### Output

Let's take a look of the generated output of Abstract segmentation.

In [10]:
pprint.pprint(output[0][1])

{'output': [{'error': 'No errors.',
             'response': [{'answer': 'Assuming that the effect of Vaporfly '
                                     'shoes is additive, the estimated '
                                     'improvements for men are between 2.0 and '
                                     '3.9 minutes, and for women are between '
                                     '0.8 and 3.5 minutes. Assuming that the '
                                     'effect of Vaporfly shoes is '
                                     'multiplicative, the estimated '
                                     'improvements for men are between 1.4 and '
                                     '2.8 percent, and for women are between '
                                     '0.6 and 2.2 percent.',
                           'context': 'We collected marathon performance data '
                                      'from a systematic sample of elite and '
                                      'sub-elite athletes

### Process the output

Let's take a look of the generated output. We need to do a little postprocessing on the raw output.

In [12]:
# Extracting context, question, and answer into a DataFrame
contexts = []
questions = []
answers = []

for item in output[0]:
    for i in item.get('output', []):
        for response in i.get('response', []):
            if any(key not in response for key in ['context', 'question', 'answer']):
                print("Missing context, question or answer in response:", response)
                continue
            
            contexts.append(response['context'])
            questions.append(response['question'])
            answers.append(response['answer'])

# Set display options
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', 1000)

df = pd.DataFrame({
    'Context': contexts,
    'Question': questions,
    'Answer': answers
})

styled_df = df.style.set_properties(**{'text-align': 'left'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'left')]
}])
styled_df

Missing context, question or answer in response: {'context': "### Ted Westling, University of Massachusetts Amherst\n_This article presents a clear and well-written study of the association between wearing the Vaporfly running shoe and race time in the context of elite marathon runners. I have several comments._\n_1. Why have separate models for women and men? Once you control for individual runner effects, doesn't this automatically include sex differences? I would expect that putting all the runners in a single model would improve power and also allow a formal assessment of the interaction of sex and Vaporfly effect via an interaction of the two._\nThis is a good question and suggestion. Our main reason for fitting separate models is that we wanted to avoid assuming that the variances of the random effects terms were the same for men and women. Looking at the table of parameter estimates, in the untransformed model, the women's standard deviations are all higher than the men's standa